In [1]:
import nltk
import pandas as pd
import numpy as np
import warnings
import re
import html
import pymorphy2
from tqdm.notebook import tqdm
warnings.filterwarnings("ignore")

In [2]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
df = positive.append(negative)

In [3]:
def del_puct(input_text):
    return re.sub('[!@#$\n-._?,]', ' ', input_text)

def del_eng(input_text):
    return re.sub('[^а-яА-Я\s]*', '', input_text)

In [4]:
df['text'] = df['text'].apply(html.unescape)
df['text'] = df['text'].apply(del_puct)
df['text'] = df['text'].apply(del_eng)
df

,text
0,хоть я и школота но поверь у нас то же са...
1,Да все таки он немного похож на него Но мой ...
2,Ну ты идиотка я испугалась за тебя
3,Кто то в углу сидит и погибает от голода ...
4,Вот что значит страшилка Но блин посмотрев...
...,...
111918,Но не каждый хочет что то исправлять
111919,скучаю так только вправляет мозги но я в...
111920,Вот и в школу в говно это идти уже надо
111921,Тауриэль не грусти обнял


In [5]:
morph = pymorphy2.MorphAnalyzer()
words_regex = re.compile('\w+')

def find_words(text, regex = words_regex):
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]


stopwords_list = nltk.corpus.stopwords.words('russian')
stopwords_add = ['всё', 'таки', 'все', 'это', 'наш', 'ещё', 'еще']

def lemmatize(words, lemmer = morph, stopwords = stopwords_list+stopwords_add):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords
            and w.isalpha()]

def preprocess(text):
    return (lemmatize(find_words(text)))

In [6]:
df.text.iloc[1]

'Да  все таки он немного похож на него  Но мой мальчик все равно лучше'

In [7]:
print(preprocess(df.text.iloc[1]))

['немного', 'похожий', 'мальчик', 'равно', 'хороший']


In [8]:
preprocessed_text = list(tqdm(map(preprocess, df['text']), total=len(df)))
df['text'] = preprocessed_text
df.sample(3)

  0%|          | 0/226834 [00:00<?, ?it/s]

,text
35804,"[спасибо, лента, особенность, гриша, столь, пр..."
91827,"[завтра, рабочий, день, пиздец, армия]"
42250,"[лекси, мочь, приставать, гвоздикойлоздика]"


In [9]:
df = df[df.text.notnull()]
df.sample(3)

,text
1027,"[балансёр, писать, неделя, боление, грипп, пох..."
13995,"[заявлять, хотеть, идти, поворот]"
65513,"[сегодня, смотреть, холод, купить, мороженое]"


In [10]:
from gensim.models import *
from gensim import corpora

In [11]:
dictionary = corpora.Dictionary(df['text'])

dictionary.filter_extremes(no_below = 3, no_above = 0.9, keep_n=None)
dictionary.save('tweet.dict')

In [12]:
corpus = [dictionary.doc2bow(text) for text in df['text']]
corpora.MmCorpus.serialize('tweet.model', corpus)

In [13]:
%time lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=20)

CPU times: total: 29.4 s
Wall time: 29.4 s


In [14]:
lda.show_topics()

[(17,
  '0.125*"блин" + 0.066*"хотеться" + 0.064*"говорить" + 0.033*"равно" + 0.027*"ранний" + 0.023*"твит" + 0.023*"грустно" + 0.023*"выйти" + 0.022*"нету" + 0.022*"еда"'),
 (1,
  '0.049*"фильм" + 0.048*"читать" + 0.037*"нога" + 0.037*"чувствовать" + 0.035*"столько" + 0.029*"сразу" + 0.029*"песня" + 0.025*"девушка" + 0.025*"мысль" + 0.024*"начать"'),
 (8,
  '0.081*"сидеть" + 0.068*"идти" + 0.052*"ночь" + 0.045*"дело" + 0.036*"ужасно" + 0.030*"смочь" + 0.028*"холодно" + 0.026*"уйти" + 0.026*"капец" + 0.022*"душа"'),
 (0,
  '0.093*"завтра" + 0.086*"спать" + 0.068*"сказать" + 0.056*"время" + 0.054*"утро" + 0.027*"обидно" + 0.026*"простить" + 0.025*"минута" + 0.024*"сегодня" + 0.022*"проснуться"'),
 (6,
  '0.074*"мама" + 0.050*"твой" + 0.044*"телефон" + 0.040*"жалко" + 0.039*"жить" + 0.030*"конец" + 0.024*"учиться" + 0.024*"класс" + 0.022*"начало" + 0.019*"соскучиться"'),
 (10,
  '0.152*"знать" + 0.062*"сделать" + 0.046*"забыть" + 0.039*"блядь" + 0.038*"казаться" + 0.034*"учить" + 0.024*"

In [15]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

%time vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

C:\Users\nikit\AppData\Roaming\Python\Python39\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


CPU times: total: 13 s
Wall time: 17.3 s
